## Qn

Given two sorted arrays nums1 and nums2 of size m and n respectively, return the median of the two sorted arrays. The overall run time complexity should be O(log (m+n)).

- Example 1:
    - Input: nums1 = [1,3], nums2 = [2]
    - Output: 2.00000
    - Explanation: merged array = [1,2,3] and median is 2.
<br><br>
- Example 2:
    - Input: nums1 = [1,2], nums2 = [3,4]
    - Output: 2.50000
    - Explanation: merged array = [1,2,3,4] and median is (2 + 3) / 2 = 2.5.
 <br><br>
- Constraints:
    - nums1.length == m
    - nums2.length == n
    - 0 <= m <= 1000
    - 0 <= n <= 1000
    - 1 <= m + n <= 2000
    - -106 <= nums1[i], nums2[i] <= 106

## Answer

In [6]:
def findMedianSortedArrays(A, B):
    l = len(A) + len(B)
    if l % 2 == 1:
        return kth(A, B, l // 2)
    else:
        return (kth(A, B, l // 2) + kth(A, B, l // 2 - 1)) / 2.   
    
def kth(a, b, k):
    print('='*50)
    print(f'a: {a} | b: {b} | k: {k}')
    if not a:
        print(f'a is empty, returning b[k]: {b[k]}')
        return b[k]
    if not b:
        print(f'b is empty, returning a[k]: {a[k]}')
        return a[k]
    
    ia, ib = len(a) // 2 , len(b) // 2 #ia is number of elements less than median of a, same for ib
    ma, mb = a[ia], b[ib]
    print(f'ia: {ia} | ib: {ib}')
    print(f'ma: {ma} | mb: {mb}')
    
    # k means: select the element where there are k values before it
    # The bigger value of ma and mb will have ia+ib+1 values below it
    
    # If the requested position k still exceeds ia+ib+1-th position, then the kth 
    # value must be to the right of both ma and mb
    if (ia + ib + 1) <= k:
        print(f'ia + ib + 1 <= k || {ia} + {ib} + 1 <= {k}')
        if ma < mb:
            print(f'ma < mb || {ma} < {mb}')
            # if ma is lower than mb, remove everything ma and below
            print(f'Returning kth(a[(ia+1):], b, k - ia - 1) kth({a[(ia+1):]}, {b}, {k-ia-1})')
            return kth(a[(ia+1):], b, k - ia - 1)
        else:
            print(f'ma >= mb || {ma} >= {mb}')
            print(f'Returning kth(a, b[(ib+1):], k - ib - 1) kth({a}, {b[(ib+1):]}, {k-ib-1})')
            return kth(a, b[(ib+1):], k - ib - 1)
    
    # If the requested position k does not exceed ia+ib+1-th position, 
    # then the kth value must be to the left of ma and mb
    else:
        print(f'ia + ib + 1 > k || {ia} + {ib} + 1 > {k}')
        if ma > mb:
            print(f'ma > mb || {ma} > {mb}')
            print(f'Returning kth(a[:(ia)], b, k) kth({a[:(ia)]}, {b}, {k})')
            return kth(a[:(ia)], b, k)
        else:
            print(f'ma <= mb || {ma} <= {mb}')
            print(f'Returning kth(a, b[:(ib)], k) kth({a}, {b[:(ib)]}, {k})')
            return kth(a, b[:(ib)], k)

In [9]:
kth([0,2,4],[1,3,5], 3)

a: [0, 2, 4] | b: [1, 3, 5] | k: 3
ia: 1 | ib: 1
ma: 2 | mb: 3
ia + ib + 1 <= k || 1 + 1 + 1 <= 3
ma < mb || 2 < 3
Returning kth(a[(ia+1):], b, k - ia - 1) kth([4], [1, 3, 5], 1)
a: [4] | b: [1, 3, 5] | k: 1
ia: 0 | ib: 1
ma: 4 | mb: 3
ia + ib + 1 > k || 0 + 1 + 1 > 1
ma > mb || 4 > 3
Returning kth(a[:(ia)], b, k) kth([], [1, 3, 5], 1)
a: [] | b: [1, 3, 5] | k: 1
a is empty, returning b[k]: 3


3

## Intro

Let the 2 arrays be `A` and `B`, with `A` of length `m` and `B` of length `n`

Note that the requested time complexity is `O(log(m+n))`. This suggests that the strategy here involves some approach where the search space is getting halved at each step.

## Brute force

Let's first think through how we can achieve this in a naive manner. 

The brute force approach is simply to concatenate the 2 arrays, sort it, and return the median index. This is the easiest to implement, but has a time complexity of O(n+m log(n+m)) instead, failing the time complexity criteria of being `O(log(m+n))`
- Concatenation: O(n+m)
- Sorting: O(n+m log(n+m))
- Return median index: O(1)

In [19]:
%%timeit
def brute_force(a: list, b: list) -> int:
    combined = a + b
    combined.sort()
    
    if len(combined) % 2 != 0:
        return combined[len(combined)//2]
    else:
        return (combined[(len(combined)//2)-1] + combined[len(combined)//2])/2

brute_force([1,1,2,3,4,5,6,7,10],[2,2,2,2,3,4,7,12,18,20])

831 ns ± 34.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Optimized Approach

Assume we are given the following 2 arrays:

```
A = [a0, a1, ... am]
B = [b0, b1, ... bn]
```

Let the length of array `A` be `m` and the length of array `B` be `n`. If `m+n` is odd, then the median value of the 2 arrays is at index `(m+n) // 2`. Else, the median value is the average of `(m+n) // 2` and `((m+n) // 2)-1`
    - For example, if m = 3 and n = 3, then median value is the average of index 3 and index 2 by the definitions above

Let's define this as a function:

In [ ]:
def get_median_value(A: list[int], B: list[int]) -> float:
    total_len = len(A) + len(B)
    if total_len % 2 != 0:
        median_value = get_value_at_index_k(A, B, total_len//2)
    else:
        median_value_1 = get_value_at_index_k(A, B, total_len//2)
        median_value_2 = get_value_at_index_k(A, B, (total_len//2) -1 )
        median_value =  (median_value_1 + median_value_2) / 2
    return median_value

def get_value_at_index_k(A: list[int], B: list[int], k: int) -> int:
    ...

We have written the definition of the median of 2 arrays into a function, which relies on some magical `get_value_at_index_k()` function to do the heavy lifting. In this case, get median value itself does not do any heavy computational lifting, so the time complexity of the overall `get_median_value()` is entirely dependent on the time complexity of `get_value_at_index_k()`

Well and good, the problem is now reduced to getting the k-th element of 2 arrays in logarithmic time. Is there a way to do this? Turns out, there is quite an intuitive way to do this through recursively halving the arrays! Let's see this using a concrete example.

```
A = [0,2,4]
B = [1,3,5]
k = 0
```

Let's suppose we want to get the 0-th element of A and B in the example above. How can we easily get rid of half of our search space? 
1. Since both arrays are sorted, we know for a fact that x_{i+1} > x_{i} for every element x in A and B. 
2. We also know that as the 0-th element, there should not be anything less than it in the combined array. 

Given these facts, let's consider the medians of both arrays. 
    - In the case when the array (either A or B) is odd, the median index is len(array)//2. 
    - When it is even, let us define it as len(array)//2 + 1
    - When len(array) == 1, then 1//2 == 0. So the element is its own median.

By definition, the median of arrays will contain elements to its left, and elements to its right, with the exception of the case when the array has length 1 (where there is nothing to its left or right), or length 2 (where there is nothing to its right). As such, the median index will tell us how many values there are to its left.
    - In array [0,1,2], the median index is 1, indicating that there is 1 value to its left
    - In array [0,1,2,3], median index is 2, indicating that there are 2 values to its left

In the example above, the median index is 1 for both A and B, indicating that there is 1 value to the left of both medians. By definition, when A and B are combined and sorted, either median A is after median B, or median B is after median A. In this case, median B exceeds median A. As a result, we conclude that median B (3) must be greater than 1 (because array B is sorted), greater than median A (2), and greater than the single value on the left of median A (0).

Since we know that there must be 3 values to the left of median B, and we wish to find the 0th element of the final array, it stands to reason that any value >= median B can be safely discarded. This brings us to the next iteration:

```
A = [0,2,4]
B = [1]
k = 0
```

We now follow the same logic. Median A is at index 1, and median B is at index 0. Comparing the 2 medians, median A (2) > median B (1), and so it exceeds 2 values (median B, and 0). As such, since we are still looking for the 0th element (which has nothing to its left), we can safely discard anything >= median A.

```
A = [0]
B = [1]
k = 0
```

Finally, we have 2 remaining values which are both at index 0. Once again, comparing the medians, we find that median B > median A. So discard anything more than or equal to median B.

```
A = [0]
B = []
k = 0
```

This gives us our final answer

That was a VERY long road to walk to get to what might seem like a pretty trivial solution at first glance. The amazing thing about this process though, is that we can identify the k-th element in any 2 arrays in logarithmic time, without having to sort the arrays! 

Let's now try it on the same array, but requesting a different value of k:

```
A = [0,2,4]
B = [1,3,5]
k = 3
```

Similar to the case above, we compare the median values of the 2 arrays. We conclude that the larger median will have at least 3 values to its left (3 is larger than 1, and also larger than 2 and 0), while the smaller median will have at most 2 values to its left (2 is larger than 0, and 2 MAY OR MAY NOT be larger than the 0th element of array B). As such, we can safely conclude that we can drop anything less than or equal to the smaller of the 2 medians.

```
A = [4]
B = [1,3,5]
k = 1
```

Note something impt that happens here; because we are deleting elements from the left of the array (as opposed to from the right in the earlier example), we must also shift the value of `k`. This is because what was the 4th element in the original combined array, must naturally have shifted left by 2 positions now that we deleted 2 values before it. i.e. if we continue to get the 4th element in the new arrays, then we will be returning the largest element of the original array (since there are only 4 values left)

Again, we compare medians. 
    - 4 has no elements on its left, while 3 has 1. And 
    - 4 is larger than 3, so it must be larger than at least 2 elements (3 and 1)
    - Hence, 4 exceeds the position we are interested in, so we eliminate it

```
A = []
B = [1,3,5]
k = 1
```

Finally, we are left with no value in array A, which leads us to taking `B[k]` as our answer, and finally gives us 3!

The algorithm works! By recursively pruning the search space, we can array at some final answer in logarithmic time. Let's try to put this in code

In [28]:
def get_value_at_index_k(A: list[int], B: list[int], k: int) -> int:
    print('='*50)
    print(f'A: {A} || B: {B}')

    ## Return the kth value of the remaining array if A or B is empty
    if not A:
        return B[k]
    if not B:
        return A[k]

    ## If both A and B are not empty, compare the medians
    median_index_A = len(A)//2
    median_index_B = len(B)//2
    median_A = A[median_index_A]
    median_B = B[median_index_B]
    print(f'median_index_A: {median_index_A} || median_index_B: {median_index_B} ')
    print(f'median_A: {median_A} || median_B: {median_B} ')

    ## To avoid confusion, let's always ensure that median A larger than median B, else you need to repeat the logic value for the case when
    ## median_B <= median_A
    if median_A < median_B:
        return get_value_at_index_k(B, A, k)

    ## The larger of median A and B will be larger than 
    ##    - all values to its left
    ##    - smaller median
    ##    - values to the left of the smaller median
    if median_A >= median_B:
        print(f'median_A >= median_B || {median_A} >= {median_B}')
        count_elements_less_than_larger_median = median_index_A + median_index_B + 1

        ## Note that there are k elements to the left of the k-th element in an array.
        ## If k is more than or equal to the count of elements to the left of the larger median, that means the true value must be at 
        ## or above the larger median
        ## Hence
        ##  - remove all elements at and below the smaller median
        ##  - remove all elements below the larger median
        ## Since we are removing from the left, subtract the number of elements removed from k
        if k >= count_elements_less_than_larger_median:
            print(f'k >= count_elements_less_than_larger_median || {k} >= {count_elements_less_than_larger_median}')
            print(f'get_value_at_index_k(A[median_index_A:], B[(median_index_B+1):], k - median_index_B - median_index_A - 1) || get_value_at_index_k({A[median_index_A:]}, {B[(median_index_B+1):]}, {k} - {median_index_B} - {median_index_A} - 1)')
            return get_value_at_index_k(A[median_index_A:], B[(median_index_B+1):], k - median_index_B - median_index_A - 1)
        
        ## If k is less than count of elements to the left of the larger median, that means the true value must be at 
        ## or below the larger median
        ## Hence, remove all elements at and above the larger median
        elif k < count_elements_less_than_larger_median:
            print(f'k < count_elements_less_than_larger_median || {k} < {count_elements_less_than_larger_median}')
            print(f'get_value_at_index_k(A[:median_index_A], B, k) || get_value_at_index_k({A[:median_index_A]}, {B}, {k})')
            return get_value_at_index_k(A[:median_index_A], B, k)

get_value_at_index_k([0,2,4,6,8], [1,3,5,7,9], 3)

A: [0, 2, 4, 6, 8] || B: [1, 3, 5, 7, 9]
median_index_A: 2 || median_index_B: 2 
median_A: 4 || median_B: 5 
A: [1, 3, 5, 7, 9] || B: [0, 2, 4, 6, 8]
median_index_A: 2 || median_index_B: 2 
median_A: 5 || median_B: 4 
median_A >= median_B || 5 >= 4
k < count_elements_less_than_larger_median || 3 < 5
get_value_at_index_k(A[:median_index_A], B, k) || get_value_at_index_k([1, 3], [0, 2, 4, 6, 8], 3)
A: [1, 3] || B: [0, 2, 4, 6, 8]
median_index_A: 1 || median_index_B: 2 
median_A: 3 || median_B: 4 
A: [0, 2, 4, 6, 8] || B: [1, 3]
median_index_A: 2 || median_index_B: 1 
median_A: 4 || median_B: 3 
median_A >= median_B || 4 >= 3
k < count_elements_less_than_larger_median || 3 < 4
get_value_at_index_k(A[:median_index_A], B, k) || get_value_at_index_k([0, 2], [1, 3], 3)
A: [0, 2] || B: [1, 3]
median_index_A: 1 || median_index_B: 1 
median_A: 2 || median_B: 3 
A: [1, 3] || B: [0, 2]
median_index_A: 1 || median_index_B: 1 
median_A: 3 || median_B: 2 
median_A >= median_B || 3 >= 2
k >= count_ele

3